<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2024notebooks/2024_0621polynomilal_fittings_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多項式回帰と多層パーセプトロン (MLP: Multi-Layered Perceptrons)  によるアンダーフィッティング，オーバーフィッテイング

- author: 浅川伸一
- date: 2024_0621


In [ ]:
%config InlineBackend.figure_format = 'retina'
# 準備作業必要なライブラリを輸入
import numpy as np
import sys
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

In [ ]:
# データの作成。ここではサイン曲線を用いる
N_sample = 100   # データ数
X = np.linspace(0, 6 * np.pi, N_sample)  # 0 から 6 π の範囲を X 入力値とする
y = np.sin(X)                            # X の範囲の正弦（サイン）関数の値を y とする

plt.figure(figsize=(8, 4))   # 表示サイズ，単位はインチ
plt.title('正弦波 (sine) 曲線')
plt.plot(X, y)               # X, y の値で図を描画
plt.show()

In [ ]:
def make_poly(points, n_ord):
    # n_ord 次の多項式を定義 ここで n_ord: 1, 2, ..., n_ord すなわち 1 次から n_ord 字までの多項式を指定
    n_points = len(points)
    data = [np.ones(n_points)]
    for n in range(n_ord):
        data.append(points ** (n+1))  # Python では記号 ** でべき乗を意味する
    return np.vstack(data).T

線形回帰，重回帰モデルは，$\displaystyle\mathbf{Y}=\mathbf{XW}$ と書くことができる。

ここで，$\mathbf{X}$  はデータ行列，$\mathbf{Y}$ は予測すべき変数行列，$\mathbf{W}$ は係数行列である。
与式を $\mathbf{W}$ について解けば，以下のようになる:

$$\begin{aligned}
\mathbf{Y} &= \mathbf{XW}\\
\mathbf{X}^{\top}\mathbf{Y} &=\mathbf{X}^{\top}\mathbf{XW}\\
\left(\mathbf{X}^{\top}\mathbf{X}\right)^{-1}\mathbf{X}^{\top}\mathbf{Y} &= \left(\mathbf{X}^{\top}\mathbf{X}\right)^{-1}\mathbf{X}^{\top}\mathbf{XW}.\\
\end{aligned}$$

従って，係数行列 $\mathbf{W}$ は次式のように書くことができる:
$$
\mathbf{W} = \left(\mathbf{X}^{\top}\mathbf{X}\right)^{-1} \mathbf{X}^{\top}\mathbf{Y}
$$

このことから，データ $\mathbf{X}$ と $\mathbf{Y}$ とが与えられたとき，回帰モデルによる予測値 $\mathbf{Y}^{\star}$ は，
$\mathbf{P}=\mathbf{X}\left(\mathbf{X}^{\top}\mathbf{X}\right)^{-1}\mathbf{X}$ とすれば，
$\displaystyle{\mathbf{\hat{Y}}}=\mathbf{PY}$ のように表すことができる。

In [ ]:
def linfit(X, Y):
    # 重回帰による近似解を返す関数
    return np.linalg.solve(np.dot(X.T,X), np.dot(X.T,Y))

なお，`numpy` では，行列の積を `.dot` で表し，行列の転置 を `.T` で表す。
従って，上のセルでは，回帰方程式 $\mathbf{X}^{\top}\mathbf{X}=\mathbf{X}^{\top}\mathbf{Y}$ を解く関数が `np.linalg.solve()` 関数である。

In [ ]:
n_samples = 10  # データから n_sample 個の点をサンプリングするために使う
n_data = len(X)

# 実際のサンプリング
# n_data 個のデータ点から n_samples 個のデータをランダムサンプリングして train_idx に保存
train_idx = np.random.choice(n_data, n_samples)

X_train = X[train_idx]   # サンプリングされた X の値
y_train = y[train_idx]   # サンプリングされた y の値

# 次行 in の後のカッコ内の数字の多項式を用いた回帰式を計算し描画
for n in (2, 3, 4, 5, 6, 7):
#for n in (6, 7, 8, 30):

    n_ord = n + 1  # プラス 1 するのはお約束
    Xtrain_poly = make_poly(X_train, n_ord)  # 多項式回帰のための準備
    w = linfit(Xtrain_poly, y_train)         # 実際の多項式に当てはめた時の回帰係数

    X_poly = make_poly(X, n_ord)             # 真の値
    y_hat = np.dot(X_poly, w)                # 予測値を y_hat として計算し保存

    plt.figure(figsize=(8, 4))               # 横 8 インチ，縦 4 インチの図の枠組みを指定

    plt.plot(X, y)                           # オリジナルのデータ曲線，サイン波の描画
    plt.plot(X, y_hat)                       # 多項回帰に基づく曲線の描画
    plt.scatter(X_train, y_train)            # サンプリングされた点の描画
    plt.title(f"{n_ord} 次式回帰")           # 図のタイトルの表示

    plt.ylim(-1.5,2.5)                       # y 軸の範囲を設定
    plt.show()

上のセルで，多項回帰の次数を変えて描画せよ。

# MLP による回帰

In [ ]:
import sys

In [ ]:
# scikit-learn から多層パーセプトロン分類器を輸入
from sklearn.neural_network import MLPRegressor

def fit_and_display(X:np.array,
                    y:np.array,
                    X_train:np.array,
                    y_train:np.array,
                    #n_samples:int,
                    n_ord:int):

    mlp = MLPRegressor()  #alpha=0.01,hidden_layer_sizes=(n_ord,),verbose=False,max_iter=1000)
    X_poly = make_poly(X_train, n_ord)
    mlp.fit(X_poly, y_train)

    X_poly_ = make_poly(X, n_ord)
    y_hat = mlp.predict(X_poly_)

    plt.figure(figsize=(8, 4))               # 横 8 インチ，縦 4 インチの図の枠組みを指定
    plt.plot(X, y)
    plt.scatter(X_train, y_train)
    plt.plot(X, y_hat)
    plt.title(f"{n_ord} 次近似")
    plt.ylim(-1.5,2.5)                       # y 軸の範囲を設定

    plt.show()

X = np.linspace(0, 6 * np.pi, N_sample)  # 0 から 6 π の範囲を X 入力値とする
y = np.sin(X)                            # X の範囲の正弦（サイン）関数の値を y とする

n_samples = 10  # データから n_sample 個の点をサンプリングするために使う
n_data = len(X)
train_idx = np.random.choice(n_data, n_samples)
X_train = np.array(X[train_idx])
y_train = np.array(y[train_idx])

for n_ord in (1,2,4,8,10):
    fit_and_display(X, y, X_train, y_train, n_ord)
